In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import timeit
import datetime
import time
import pprint
import itertools
import pickle
import sklearn
import dask.dataframe as dd
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
import fastparquet
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
# Distances
import scipy.spatial.distance as distance
# KL
from scipy.stats import entropy
# Normalize
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Pairwise distances
from sklearn.metrics.pairwise import pairwise_distances
import h5py

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [8]:
ts = pd.read_pickle("DataStore/2018-07-P2/targ_samp_0717.pkl")
# ts = pd.read_pickle("DataStore/2018-07-P2/targ_samp_0717.pkl")[["tp", "tp_primclass", "tp_gyear", "tp_inv_msa"]]
ldf = pd.read_csv("RawData/Cleaned/patent_lawyer.csv", index_col=0)
ldf = ldf.loc[ldf["patent"].isin(ts["tp"]) | ldf["patent"].isin(ts["cp"])]
%time ldf = {n:g["lawyer_id"].tolist() for n,g in ldf[["patent", "lawyer_id"]].groupby("patent")}

# Lawyer match
l_match = (set(ldf.get(tp, [])).intersection(set(ldf.get(op, []))) for tp, op in zip(ts["tp"], ts["cp"]))
ts["lawyer_match"] = [True if len(i) >=1 else False for i in l_match ]

In [9]:
ts["lawyer_match"].value_counts()

False    281526
True        553
Name: lawyer_match, dtype: int64

In [13]:
ts[["tp_match_10", "cp_match_10", "lawyer_match"]].groupby("lawyer_match").mean()

,tp_match_10,cp_match_10
lawyer_match,,
False,0.111619,0.045162
True,0.133332,0.087529


In [11]:
ts.head()

,tp,cp,tp_primclass,tp_gyear,tp_inv_msa,cp_gyear,cp_inv_msa,tp_match_5,cp_match_5,tp_match_10,cp_match_10,cite_msa_match_5,cite_msa_diff_5,cite_msa_match_10,cite_msa_diff_10,lawyer_match
3,5241648,5241671,1.0,1993,"San Jose-Sunnyvale-Santa Clara, CA",1993,"San Diego-Carlsbad-San Marcos, CA",0.200000,0.064516,0.285714,0.151261,[5448727],"[5666526, 5537589, 5664172, 5594898]","[6505189, 6374232, 5448727, 6167399]","[6643636, 6640221, 5983215, 5666526, 6064999, ...",False
5,6665662,6665663,1.0,2003,"San Francisco-Oakland-Fremont, CA",2003,"San Jose-Sunnyvale-Santa Clara, CA",0.000000,0.500000,0.145833,0.600000,NaN,"[7131069, 7243095, 7376641, 7284235, 7441007, ...","[8560534, 7752159, 7756810, 8271794, 8495002, ...","[8205242, 7831559, 7131069, 8307007, 7243095, ...",False
9,5797117,5797128,1.0,1998,"Los Angeles-Long Beach-Santa Ana, CA",1998,"Colorado Springs, CO",0.000000,0.000000,0.000000,0.025641,NaN,"[6092073, 5897633, 6654879, 5903895, 5987253]",NaN,"[6092073, 5897633, 6654879, 5903895, 5987253]",False
11,7502777,7502779,1.0,2009,"San Jose-Sunnyvale-Santa Clara, CA",2009,"Austin-Round Rock-San Marcos, TX",0.000000,0.000000,0.000000,0.000000,NaN,"[7784033, 7921084]",NaN,"[7784033, 7921084]",False
15,7599939,7599941,1.0,2009,"San Jose-Sunnyvale-Santa Clara, CA",2009,"Los Angeles-Long Beach-Santa Ana, CA",0.166667,0.150000,0.250000,0.103448,[8903836],"[8578393, 8079081, 8156553, 8407335, 8719232]","[8903836, 9298691]","[8578393, 8079081, 9657567, 8156553, 8407335, ...",False
